In [ ]:
# Algorithm 1: Four-query Clifford Tester
# https://www.arxiv.org/abs/2510.07164 pg. 21

from qiskit import QuantumCircuit

from lib.clifford_tester import clifford_tester_paired_runs, get_clifford_tester_circuit
from lib.clifford_tester.results import PairedRawResults, PairedSample


def acceptance_rate(U, n, shots=100):
    raw_dicts = clifford_tester_paired_runs(U, n, shots=shots)
    results = PairedRawResults(samples=[PairedSample(**d) for d in raw_dicts])
    return results.summarise()

In [ ]:
# Test with Clifford gates (should accept with high probability)
print("Testing Clifford gates:")

# Hadamard (Clifford)
H = QuantumCircuit(1)
H.h(0)
print(f"  Hadamard acceptance rate: {acceptance_rate(H, 1):.2f}")

# S gate (Clifford)
S = QuantumCircuit(1)
S.s(0)
print(f"  S gate acceptance rate: {acceptance_rate(S, 1):.2f}")

# CNOT (Clifford, 2-qubit)
CNOT = QuantumCircuit(2)
CNOT.cx(0, 1)
print(f"  CNOT acceptance rate: {acceptance_rate(CNOT, 2):.2f}")

# Identity (Clifford)
I = QuantumCircuit(1)
print(f"  Identity acceptance rate: {acceptance_rate(I, 1):.2f}")

In [ ]:
# Test with non-Clifford gates (should reject with noticeable probability)
print("\nTesting non-Clifford gates:")

# T gate (non-Clifford)
T = QuantumCircuit(1)
T.t(0)
print(f"  T gate acceptance rate: {acceptance_rate(T, 1):.2f}")

# Rotation by arbitrary angle (non-Clifford)
Rx = QuantumCircuit(1)
Rx.rx(0.3, 0)  # Arbitrary rotation angle
print(f"  Rx(0.3) acceptance rate: {acceptance_rate(Rx, 1):.2f}")

# Toffoli (non-Clifford, 3-qubit)
Toffoli = QuantumCircuit(3)
Toffoli.ccx(0, 1, 2)
print(f"  Toffoli acceptance rate: {acceptance_rate(Toffoli, 3):.2f}")

In [ ]:
N = 4

# Test with larger gates to check simulator performance
print(f"\nTesting {N}-qubit gates:")

# N-qubit Clifford: H on all qubits + chain of CNOTs
N_qubit_clifford = QuantumCircuit(N)
for i in range(N):
    N_qubit_clifford.h(i)
for i in range(N - 1):
    N_qubit_clifford.cx(i, i + 1)
print(N_qubit_clifford.draw())
print(f"  {N}-qubit Clifford (H + CNOT chain) acceptance rate: {acceptance_rate(N_qubit_clifford, N, shots=10):.2f}")

# 7-qubit non-Clifford: T gates on all qubits
N_qubit_non_clifford = QuantumCircuit(N)
for i in range(N):
    N_qubit_non_clifford.t(i)
print(N_qubit_non_clifford.draw())
print(f"  {N}-qubit non-Clifford (T gates) acceptance rate: {acceptance_rate(N_qubit_non_clifford, N, shots=10):.2f}")

In [ ]:
# == Drawing out tester circuits ==
import matplotlib.pyplot as plt
from IPython.display import display

tester_circuit_to_view = get_clifford_tester_circuit(H, 1, (1, 0))

print("Entire Weyl-Choi state")
display(tester_circuit_to_view.draw("mpl"))
plt.close()

print("Bell state + Weyl operator")
display(tester_circuit_to_view.decompose(reps=1).draw("mpl"))
plt.close()

print("Base gates")
display(tester_circuit_to_view.decompose(reps=2).draw("mpl"))
plt.close()